# YOLOv8 License Plate Detection

In [ ]:
# missing yolo dep
!pip install lapx>=0.5.2
# OCR library
!pip install easyocr

In [1]:
import ast
import cv2 as cv
import easyocr
from glob import glob
import numpy as np
import pandas as pd
import string
from ultralytics import YOLO

## License Plate Detection

As seen during the training - the model, just using the COCO training weights, is very capable of detecting cars, trucks and buses. But number plates seem to be a bit harder - the model often confuses street signs or just basic backgound noise as a car registration plate. The positive is that it rarely misses a plate.

<br/>

We can optimize the process by combining the power of two models - a regular COCO-trained version of YOLOv8 and our number plate detector. Only if the first one detects a car we want the detector to start searching for a plate within the generated bounding box.

<br/>

* This guide is based on the [DeepSORT & EasyOCR Repository](https://github.com/computervisioneng/automatic-number-plate-recognition-python-yolov8) by [@computervisioneng](https://github.com/computervisioneng). But I replaced the [DeepSORT Dependency](https://github.com/abewley/sort) with the YOLOv8 included Track function.

In [2]:
# regular pre-trained yolov8 model for car recognition
# coco_model = YOLO('yolov8n.pt')
coco_model = YOLO('yolov8s.pt')
# yolov8 model trained to detect number plates
np_model = YOLO('runs/detect/train4/weights/best.pt')

In [3]:
# read in test video paths
videos = glob('inputs/*.mp4')
print(videos)

['inputs/uk_dash_1.mp4', 'inputs/uk_dash_2.mp4']


### STEP 1 Implementing the Car Detection

Get the bounding boxes of all vehicles in your video recording with prediction confidence score and object tracking ID

In [ ]:
# read video by index
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
# all vehicle class IDs from the COCO dataset (car, motorbike, truck) https://docs.ultralytics.com/datasets/detect/coco/#dataset-yaml
vehicles = [2,3,5]
vehicle_bounding_boxes = []

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        # use track() to identify instances and track them frame by frame
        detections = coco_model.track(frame, persist=True)[0]
        # save cropped detections
        # detections.save_crop('outputs')
        # print nodel predictions for debugging
        # print(results)

        for detection in detections.boxes.data.tolist():
            # print detection bounding boxes for debugging
            # print(detection)
            x1, y1, x2, y2, track_id, score, class_id = detection
            # I am only interested in class IDs that belong to vehicles
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])

# print found bounding boxes for debugging
print(vehicle_bounding_boxes)
video.release()

This code now collects all vehicle bounding boxes from the video and writes them into the `vehicle_bounding_boxes` list. Besides the bbox coordinates this list also contains the tracking ID of the detected vehicle - they should stay the same frame-to-frame for every detected vehicle and serve as a unique identifier. And the score - how confident is the model that this bbox acutally contains a vehicle with calues from `0`-`1`:


> `[[762.6422729492188, 614.1869506835938, 1121.368408203125, 911.6900024414062, 1.0, 0.9254793524742126], [1196.008056640625, 635.3404541015625, 1526.3975830078125, 828.6062622070312, 2.0, 0.8488578200340271], [1672.98193359375, 613.9304809570312, 1912.382080078125, 819.3222045898438, 3.0, 0.5385741591453552], [758.7203369140625, 612.6467895507812, 1119.0892333984375, 917.7677612304688, 1.0, 0.925308883190155], [1195.9505615234375, 635.4146118164062, 1527.97412109375, 830.3245239257812, 2.0, 0.865635871887207], [1692.5439453125, 613.0050659179688, 1917.7542724609375, 819.2852783203125, 3.0, 0.5493771433830261], [754.7435302734375, 612.3658447265625, 1115.0045166015625, 919.653076171875, 1.0, 0.9127519130706787], [1194.00341796875, 634.9168090820312, 1527.029541015625, 832.383544921875, 2.0, 0.8814489841461182], [1688.8155517578125, 615.6485595703125, 1920.0, 812.7891235351562, 3.0, 0.6132197976112366], [752.8799438476562, 611.2362060546875, 1111.976318359375, 920.200927734375, 1.0, 0.9137689471244812], [1192.805908203125, 634.3713989257812, 1526.1273193359375, 832.46337890625, 2.0, 0.8671290278434753], [1680.4443359375, 616.384033203125, 1920.0, 813.3687744140625, 3.0, 0.6371273994445801], [750.9274291992188, 611.5806884765625, 1110.2657470703125, 915.3110961914062, 1.0, 0.9381350874900818], [1189.63916015625, 634.7803955078125, 1525.4072265625, 833.2440185546875, 2.0, 0.888615071773529], [1669.8206787109375, 616.796142578125, 1920.0, 808.6288452148438, 3.0, 0.5068169236183167], [748.747802734375, 609.5638427734375, 1109.0101318359375, 912.808837890625, 1.0, 0.9158740639686584], [1187.832275390625, 634.11328125, 1524.633056640625, 832.628173828125, 2.0, 0.8583219647407532], [1659.7103271484375, 615.9025268554688, 1920.0, 823.25048828125, 3.0, 0.7755634784698486], [745.1077270507812, 609.5160522460938, 1107.8214111328125, 912.8062133789062, 1.0, 0.9354495406150818], [1186.91455078125, 634.5582885742188, 1524.004150390625, 832.4244995117188, 2.0, 0.8758277297019958], [1650.2227783203125, 613.749267578125, 1920.0, 828.9586791992188, 3.0, 0.7407982349395752], [742.2940673828125, 610.4445190429688, 1106.597900390625, 912.6227416992188, 1.0, 0.9281387329101562], [1186.1158447265625, 634.223876953125, 1523.3406982421875, 832.6515502929688, 2.0, 0.8710047006607056], [1638.47705078125, 614.6183471679688, 1919.968017578125, 833.5314331054688, 3.0, 0.8480165600776672], [741.3974609375, 610.8768920898438, 1105.543701171875, 912.5601806640625, 1.0, 0.9410984516143799], [1185.1246337890625, 633.4691162109375, 1523.3682861328125, 832.612060546875, 2.0, 0.8842733502388], [1627.5872802734375, 616.9085693359375, 1919.9117431640625, 829.2400512695312, 3.0, 0.85666424036026], [741.3576049804688, 610.9183959960938, 1103.5601806640625, 914.4734497070312, 1.0, 0.9404377937316895], [1183.273681640625, 633.708984375, 1522.5953369140625, 833.3422241210938, 2.0, 0.8721591234207153], [1618.3934326171875, 619.4539794921875, 1919.864013671875, 827.6344604492188, 3.0, 0.8759608864784241]]`


Using the `save_crop()` function shows me that the first 10 frames of my video contain three different cars:

![YOLOv8 License Plate Detection](./outputs/detected_cars.webp)

### STEP 2 Implementing the License Plate Detection

Use the bounding box for each vehicle and use the number plate detector model to try to find the corresponding plate within in the confinement of those boxes.

In [ ]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # debugging check if bbox lines up with detected vehicles (should be identical to save_crops() above
                    # cv.imwrite(str(track_id) + '.jpg', roi)
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # verify detections
                        print(license_plate, 'track_id: ' + str(bbox[4]))
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        cv.imwrite(str(track_id) + '.jpg', plate)
                        
video.release()

By using the tracking ID I can make sure that every license plate - as seen above the video contained several instances of the same 3 cars - is only returned ones:

![YOLOv8 License Plate Detection](./outputs/detected_plates.webp)

### STEP 3 Preprocess License Plates

In [ ]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        cv.imwrite(str(track_id) + '_gray.jpg', plate_gray)
                        cv.imwrite(str(track_id) + '_thresh.jpg', plate_treshold)
                        
video.release()

![YOLOv8 License Plate Detection](./outputs/detected_plates_processed.webp)

### STEP 4 Read License Plates

In [4]:
# Initialize the OCR reader
reader = easyocr.Reader(['en'], gpu=True)

In [5]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')
        
        return text, score

    return None, None

In [6]:
def write_csv(results, output_path):
    
    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{},{}\n'.format(
            'frame_number', 'track_id', 'car_bbox', 'car_bbox_score',
            'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number',
            'license_text_score'))

        for frame_number in results.keys():
            for track_id in results[frame_number].keys():
                print(results[frame_number][track_id])
                if 'car' in results[frame_number][track_id].keys() and \
                   'license_plate' in results[frame_number][track_id].keys() and \
                   'number' in results[frame_number][track_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{},{}\n'.format(
                        frame_number,
                        track_id,
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['car']['bbox'][0],
                            results[frame_number][track_id]['car']['bbox'][1],
                            results[frame_number][track_id]['car']['bbox'][2],
                            results[frame_number][track_id]['car']['bbox'][3]
                        ),
                        results[frame_number][track_id]['car']['bbox_score'],
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['license_plate']['bbox'][0],
                            results[frame_number][track_id]['license_plate']['bbox'][1],
                            results[frame_number][track_id]['license_plate']['bbox'][2],
                            results[frame_number][track_id]['license_plate']['bbox'][3]
                        ),
                        results[frame_number][track_id]['license_plate']['bbox_score'],
                        results[frame_number][track_id]['license_plate']['number'],
                        results[frame_number][track_id]['license_plate']['text_score'])
                    )
        f.close()

In [ ]:
results = {}

# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        
                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, './results.csv')
video.release()

This returns a list with bounding box metrics for every frame with a successful detection:

<br/>

| frame_number | track_id | car_bbox | license_plate_bbox | license_plate_bbox_score | license_number | license_number_score |
| -- | -- | -- | -- | -- | -- | -- |
| 0 | 1.0 | [760.1986694335938 614.2100830078125 1123.09130859375 914.9498901367188] | [110.20427703857422 133.25326538085938 238.5574493408203 175.96791076660156] | 0.7692280411720276 | BPG6UXN | 0.7290849695998655 |
| 1 | 1.0 | [758.7349243164062 612.4984741210938 1122.470458984375 919.1956787109375] | [109.57369995117188 134.78448486328125 238.8947296142578 179.6195831298828] | 0.767607569694519 | BP6EUXN | 0.27891552972114064 |
| 2 | 1.0 | [755.6078491210938 612.161865234375 1118.7542724609375 920.3657836914062] | [109.76798248291016 134.661376953125 239.85276794433594 180.43345642089844] | 0.7666334509849548 | BP66UXN | 0.7696779876170268 |
| 3 | 1.0 | [753.9749755859375 611.0296630859375 1115.607421875 920.6179809570312] | [109.80683898925781 134.79702758789062 239.79380798339844 180.0568389892578] | 0.7609436511993408 | BPG6UXN | 0.5947437696221942 |

### STEP 5 Clean-Up License Plate Format

In [7]:
# Mapping dictionaries for character conversion
# characters that can easily be confused can be 
# verified by their location - an `O` in a place
# where a number is expected is probably a `0`
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

In [8]:
def license_complies_format(text):
    # True if the license plate complies with the format, False otherwise.
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

In [9]:
def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

In [10]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        # verify that text is conform to a standard license plate
        if license_complies_format(text):
            # bring text into the default license plate format
            return format_license(text), score

    return None, None

In [ ]:
results = {}

# read video by index
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the entire video
while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        
                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, './results.csv')
video.release()

In [40]:
results = pd.read_csv('./results.csv')

# show results for tracking ID `1` - sort by OCR prediction confidence
results[results['track_id'] == 1.].sort_values(by='license_text_score', ascending=False)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score
175,839,298.0,[775.8486938476562 504.52294921875 1095.532592...,0.925278,[102.20135498046875 212.2305908203125 218.7746...,0.752586,NL60GXO,0.988261
29,50,298.0,[846.99609375 521.3043823242188 1254.532104492...,0.931958,[133.1925506591797 275.73577880859375 280.6121...,0.740573,NL60GXO,0.966773
146,799,298.0,[810.77734375 522.0484008789062 1130.535888671...,0.914011,[102.2442626953125 215.42474365234375 218.7385...,0.752845,NL60GXO,0.953542
147,800,298.0,[810.7708740234375 521.808349609375 1130.57128...,0.912922,[102.17294311523438 215.99032592773438 218.767...,0.754186,NL60GXO,0.953522
284,1337,298.0,[843.4232788085938 523.5321044921875 1257.2657...,0.910718,[163.98861694335938 263.2216796875 300.1403503...,0.757695,NL60GXO,0.934405
...,...,...,...,...,...,...,...,...
191,1010,298.0,[865.4359741210938 488.0260009765625 1060.5764...,0.861625,[65.1905517578125 123.86817169189453 130.26571...,0.761225,KL60GZO,0.043224
355,1462,298.0,[685.121826171875 514.077880859375 888.8001098...,0.832969,[92.80020904541016 110.36637115478516 153.4690...,0.739499,HI60CIO,0.036080
392,2306,298.0,[462.48388671875 512.8485717773438 933.4752197...,0.929456,[121.44440460205078 294.94183349609375 269.183...,0.722692,WL60YNL,0.031725
517,2684,298.0,[856.17333984375 514.7470703125 1043.54296875 ...,0.887135,[59.788631439208984 116.58961486816406 126.729...,0.738799,HL60CKD,0.030968


### STEP 6 Visualize the Results

In [34]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=6, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img

In [43]:
# read video by index
video = cv.VideoCapture(videos[1])

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'DIVX')
out = cv.VideoWriter('./outputs/processed.avi', fourcc, 20.0, size)

# reset video before you re-run cell below
frame_number = -1
video.set(cv.CAP_PROP_POS_FRAMES, 0)

True

In [44]:
ret = True

while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        df_ = results[results['frame_number'] == frame_number]
        for index in range(len(df_)):
            # draw car
            vhcl_x1, vhcl_y1, vhcl_x2, vhcl_y2 = ast.literal_eval(df_.iloc[index]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            
            draw_border(
                frame, (int(vhcl_x1), int(vhcl_y1)),
                (int(vhcl_x2), int(vhcl_y2)), (0, 255, 0),
                12, line_length_x=200, line_length_y=200)
            
            # draw license plate
            plate_x1, plate_y1, plate_x2, plate_y2 = ast.literal_eval(df_.iloc[index]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

            # region of interest
            roi = frame[int(vhcl_y1):int(vhcl_y2), int(vhcl_x1):int(vhcl_x2)]
            cv.rectangle(roi, (int(plate_x1), int(plate_y1)), (int(plate_x2), int(plate_y2)), (0, 0, 255), 6)

            # write detected number
            (text_width, text_height), _ = cv.getTextSize(
                df_.iloc[index]['license_plate_number'],
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                6)

            cv.putText(
                frame,
                df_.iloc[index]['license_plate_number'],
                (int((vhcl_x2 + vhcl_x1 - text_width)/2), int(vhcl_y1 - text_height)),
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                (0, 255, 0),
                6
            )

        out.write(frame)
        frame = cv.resize(frame, (1280, 720))

out.release()
video.release()

![processed.webp](./outputs/processed.webp)